In [3]:
import tensorflow as tf
import numpy as np
import collections
import os
import argparse
import datetime as dt

from collections import Counter
from random import random
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu

In [4]:
train_sentences = [line.strip() for line in open("data/fr/train.txt").readlines()]
val_sentences = [line.strip() for line in open("data/fr/valid.txt").readlines()]
test_sentences = [line.strip() for line in open("data/fr/test.txt").readlines()]

train_sentences = [x for x in train_sentences if x] 
val_sentences = [x for x in val_sentences if x] 
test_sentences = [x for x in test_sentences if x] 
print(len(train_sentences))
print(len(val_sentences))
print(len(test_sentences))

print(train_sentences[0])
print(train_sentences[1])
print(train_sentences[2])

33997
3003
3003
reprise de la session
je déclare reprise la session du parlement européen qui avait été interrompue le vendredi 00 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances .
comme vous avez pu le constater , le grand " bogue de l'an 0000 " ne s'est pas produit . en revanche , les citoyens d'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles .


In [5]:
#german pop
'''
sentences.pop(23479)
sentences.pop(23479)
sentences.pop(23479)
sentences.pop(36324)
'''

'''
#russian pop
sentences.pop(3098)
sentences.pop(31275)
'''

sentences = train_sentences
# Lower-case the sentence, tokenize them and add <SOS> and <EOS> tokens
sentences = [["<SOS>"] + word_tokenize(sentence.lower()) + ["<EOS>"] for sentence in sentences]

print("done")


done


In [6]:
for ind,sen in enumerate(sentences):
    for i in range(20):
        try:
            sen.remove("<")
            sen.remove("unk")
        except:
            pass
        
print(sentences[2])

['<SOS>', 'comme', 'vous', 'avez', 'pu', 'le', 'constater', ',', 'le', 'grand', '``', 'bogue', 'de', "l'an", '0000', '``', 'ne', "s'est", 'pas', 'produit', '.', 'en', 'revanche', ',', 'les', 'citoyens', "d'un", 'certain', 'nombre', 'de', 'nos', 'pays', 'ont', 'été', 'victimes', 'de', 'catastrophes', 'naturelles', 'qui', 'ont', 'vraiment', 'été', 'terribles', '.', '<EOS>']


In [ ]:
s = sentences[31275]
print(s)

In [ ]:
'''
#vocab german
vocabulary = ["<SOS>"] + ["a"] + ["ä"] + ["á"] + ["à"] + ["b"] + ["c"] + ["d"] + ["e"] + ["ê"] + ["é"] + ["è"] + \
["ë"] + ["f"] + ["g"] + ["h"] + ["i"] + ["¡"] + ["ï"] + ["j"] + ["k"] + ["l"] + \
["m"] + ["n"] + ["ñ"] + ["o"] + ["ó"] + ["ò"] + ["ô"] + ["í"] + \
["ö"] + ["p"] + ["q"] + ["r"] + ["s"] + ["t"] + ["u"] + ["ü"] + ["ú"] + ["û"] + ["v"] + ["w"] + \
["x"] + ["y"] + ["z"] + ["<EOW>"] + ["<EOS>"] + [">"] + ["-"] + ["."] + ["'"] + ["`"] + ["``"] + ["0"] + ["1"] + ["2"] + ["3"] + \
["4"] + ["5"] + ["6"] + ["7"] + ["8"] + ["9"] + ["&"] + ["<"] + ["$"] + ["#"] + ["/"] + [","] + ["|"] + \
["@"] + ["%"] + ["^"] + ["\\"] + ["*"] + ["("] + [")"] + ["{"] + ["}"] + [":"] + [";"] + ["ß"] + ["?"] + ["!"] + \
["\xad"] + ["ø"] + ["ç"] + ["+"] + ["æ"] + ["["] + ["]"] + ["μ"] + ["å"] + ["'̧"]

max_char_len = 604
'''

'''
#vocab spanish
vocabulary = ["<SOS>"] + ["a"] + ["á"] + ["ä"] + ["à"] + ["b"] + ["c"] + ["d"] + ["e"] + ["é"] + ["ê"] + \
["è"] + ["f"] + ["g"] + ["h"] + ["i"] + ["¡"] + ["ï"] + ["j"] + ["k"] + ["l"] + ["å"] + \
["í"] +["m"] + ["n"] + ["ñ"] + ["o"] + \
["ó"] + ["ö"] + ["ô"] + ["ò"] + ["p"] + ["q"] + ["r"] + ["s"] + ["t"] + ["u"] + ["ú"] + ["ü"] + ["û"] + ["v"] + ["w"] + \
["x"] + ["y"] + ["z"] + ["<EOW>"] + ["<EOS>"] + [">"] + ["-"] + ["."] + ["'"] + ["`"] + ["``"] + ["0"] + ["1"] + ["2"] + ["3"] + \
["4"] + ["5"] + ["6"] + ["7"] + ["8"] + ["9"] + ["&"] + ["<"] + ["$"] + ["#"] + ["/"] + [","] + ["|"] + \
["@"] + ["%"] + ["^"] + ["\\"] + ["*"] + ["("] + [")"] + ["{"] + ["}"] + [":"] + [";"] + ["¿"] + ["?"] + ["["] + \
["]"] + ["!"] + ["ø"] + ["ç"] + ["\xad"] + ["+"] + ["μ"] + ["√"] + ["ß"] + ["æ"]

max_char_len = 520

'''

'''
#vocab russian
vocabulary = ["<SOS>"] + ["а"] + ["b"] + ["с"] + ["d"] + ["е"] + ["f"] + ["в"] + ["к"] + ["х"] + ["ё"] + \
["g"] + ["h"] + ["i"] + ["j"] + ["k"] + ["l"] + ["m"] + ["n"] + ["о"] + ["н"] + ["я"] + \
["р"] + ["q"] + ["r"] + ["s"] + ["t"] + ["u"] + ["v"] + ["w"] + ["з"] + ["г"] + ["т"] + ["м"] + \
["x"] + ["у"] + ["z"] + ["<EOW>"] + ["<EOS>"] + [">"] + ["-"] + ["."] + ["'"] + ["0"] + ["1"] + ["2"] + ["3"] + \
["4"] + ["5"] + ["6"] + ["7"] + ["8"] + ["9"] + ["&"] + ["<"] + ["$"] + ["#"] + ["/"] + [","] + ["|"] + \
["@"] + ["%"] + ["^"] + ["\\"] + ["*"] + ["("] + [")"] + ["{"] + ["}"] + [":"] + [";"] + ["и"] + ["ч"] + ["л"] + ["д"] + \
["п"] + ["ц"] + ["ь"] + ["ы"] + ["б"] + ["щ"] + ["ш"] + ["э"] + ["ф"] + ["й"] + ["?"] + ["ж"] + ["ю"] + ["ъ"] + ["`"] + \
["!"] + ["["] + ["]"] + ["<"] + ["ц"] + ["+"] + ["="]

max_char_len = 624
'''

'''
#vocab french 
vocabulary = ["<SOS>"] + ["a"] + ["ä"] + ["á"] + ["à"] + ["â"]+ ["b"] + ["c"] + ["d"] + ["e"] + ["ê"] + ["é"] + ["è"] + \
["ë"] + ["f"] + ["g"] + ["h"] + ["i"] + ["¡"] + ["ï"] + ["j"] + ["k"] + ["l"] + \
["m"] + ["n"] + ["ñ"] + ["o"] + ["ó"] + ["ò"] + ["ô"] + ["í"] + ["î"] + \
["ö"] + ["p"] + ["q"] + ["r"] + ["s"] + ["t"] + ["u"] + ["ü"] + ["ú"] + ["ù"] + ["û"] + ["v"] + ["w"] + \
["x"] + ["y"] + ["z"] + ["<EOW>"] + ["<EOS>"] + [">"] + ["-"] + ["."] + ["'"] + ["`"] + ["``"] + ["0"] + ["1"] + ["2"] + ["3"] + \
["4"] + ["5"] + ["6"] + ["7"] + ["8"] + ["9"] + ["&"] + ["<"] + ["$"] + ["#"] + ["/"] + [","] + ["|"] + \
["@"] + ["%"] + ["^"] + ["\\"] + ["*"] + ["("] + [")"] + ["{"] + ["}"] + [":"] + [";"] + ["ß"] + ["?"] + ["!"] + \
["\xad"] + ["ø"] + ["ç"] + ["+"] + ["æ"] + ["["] + ["]"] + ["μ"] + ["å"] + ["'̧"]

max_char_len = 524
'''

In [7]:

vocabulary = ["<SOS>"] + ["a"] + ["ä"] + ["á"] + ["à"] + ["â"]+ ["b"] + ["c"] + ["d"] + ["e"] + ["ê"] + ["é"] + ["è"] + \
["ë"] + ["f"] + ["g"] + ["h"] + ["i"] + ["¡"] + ["ï"] + ["j"] + ["k"] + ["l"] + \
["m"] + ["n"] + ["ñ"] + ["o"] + ["ó"] + ["ò"] + ["ô"] + ["í"] + ["î"] + \
["ö"] + ["p"] + ["q"] + ["r"] + ["s"] + ["t"] + ["u"] + ["ü"] + ["ú"] + ["ù"] + ["û"] + ["v"] + ["w"] + \
["x"] + ["y"] + ["z"] + ["<EOW>"] + ["<EOS>"] + [">"] + ["-"] + ["."] + ["'"] + ["`"] + ["``"] + ["0"] + ["1"] + ["2"] + ["3"] + \
["4"] + ["5"] + ["6"] + ["7"] + ["8"] + ["9"] + ["&"] + ["<"] + ["$"] + ["#"] + ["/"] + [","] + ["|"] + \
["@"] + ["%"] + ["^"] + ["\\"] + ["*"] + ["("] + [")"] + ["{"] + ["}"] + [":"] + [";"] + ["ß"] + ["?"] + ["!"] + \
["\xad"] + ["ø"] + ["ç"] + ["+"] + ["æ"] + ["["] + ["]"] + ["μ"] + ["å"] + ["'̧"]

vocabulary_size = len(vocabulary)

token2index = {token:index for index,token in enumerate(vocabulary)}
index2token = {index:token for index,token in enumerate(vocabulary)}
one_hot_embeddings = np.eye(vocabulary_size)
print(token2index.get("*"))
print(index2token.get(1))
print(one_hot_embeddings[token2index.get("\\")])

77
a
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


In [8]:
max_word_length = 0
maxid = 0
for i in range(len(sentences)):
    l = len(sentences[i])
    if l > max_word_length:
        maxid = i
        max_word_length = l
        
print(max_word_length) 
print(maxid)

83
23648


In [29]:
def embed_producer(sentences):
    max_char_len = 524
    s_tensor = np.empty((len(sentences),max_char_len,vocabulary_size))
    word_loc_all = np.zeros((len(sentences),max_word_length))
    eow_loc_all = np.zeros((len(sentences),max_char_len))
    
    #for s in sentences[23478:23480]:
    for i in range(len(sentences)):
        s = sentences[i]
        embed = np.zeros((max_char_len,vocabulary_size))
        word_loc = np.zeros(max_word_length)
        eow_loc = np.zeros(max_char_len)
        prev = 0
        count = 0 
        #print(i)
        for k in range(len(s)):
            w = s[k]
            #print(w)
            for id,token in enumerate(w):
                
                if (w == "<EOS>") | (w == "<SOS>") | (w == ">"):
                    break
                else:
                    #print(prev + id)
                    #print(token)
                    count+=1
                    embed[prev + id,:] = np.squeeze(one_hot_embeddings[token2index.get(token)])
                
            if (w == "<EOS>") | (w == "<SOS>"):
                word_loc[k] = id + 1
                #print(prev)
                embed[prev,:] = one_hot_embeddings[token2index.get(w)]
                count +=1
                eow_loc[count] = 1
                prev = prev + id + 1 
                
            elif (w == ">"):
                word_loc[k] = id + 1
                count +=1
                embed[prev,:] = one_hot_embeddings[token2index.get(w)]
                prev = prev + id + 1 
                embed[prev,:] = one_hot_embeddings[token2index.get("<EOW>")]
                count +=1
                eow_loc[count] = 1
                prev = prev + 1
                
            else: 
                prev = prev + id + 1
                word_loc[k] = id + 1 
                #print(prev)
                embed[prev,:] = one_hot_embeddings[token2index.get("<EOW>")]
                count +=1 
                eow_loc[count] = 1
                prev = prev + 1
                
            
        s_tensor[i,:,:] = embed
        eow_loc_all[i,:] = eow_loc
        
        
        #to get word end locations to retrieve hidden states later 
        word_loc_all[i,0] = word_loc[0]
        for j in range(1,len(s)):
            word_loc_all[i,j] = word_loc_all[i,j-1] + word_loc[j]
            
        
    return s_tensor,word_loc_all,eow_loc_all 


In [30]:
data,word_loc_all,eow_loc_all = embed_producer(sentences)
print("done")

done


In [ ]:
print(sentences[4607])
print(word_loc_all[4607])
print(eow_loc_all[4607])